In [1]:

!pip install langchain-google-genai
!pip install langchain-huggingface
!pip install pypdf
!pip install langchain-community
!pip install langchain-chroma
!pip install langchain
!pip install langchain-text-splitters
!pip install -U pydantic
!pip install huggingface_hub==0.24.7
!pip install jq
!pip install transformers
!pip install faiss-gpu
!pip install torch

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [7]:
from datasets import load_dataset

ds = load_dataset("HPAI-BSC/medqa-cot")

In [8]:
medqa_save = []

In [9]:
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("ncbi/MedCPT-Article-Encoder")
tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Article-Encoder")


index = faiss.IndexFlatIP(768)


In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [97]:
import os
medqa_embed = []
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

model = model.to(device)

In [98]:
for i in range(len(ds["train"])):
    if i % 1000 == 0:
        print(f"i: {i}")
    data = ds["train"][i]
    question = data["question"]
    answer = data["response"]
    # medqa_save.append([question, answer])
    try:
        with torch.no_grad():
            encoded = tokenizer(
                [[question, answer]], 
                truncation=True, 
                padding=True, 
                return_tensors='pt',
                max_length=512,
            ).to(device)
            embeds = model(**encoded).last_hidden_state[:, 0, :]
            medqa_embed.append(embeds[0].cpu().numpy())
    except Exception:
        print(f"err: {i}")
        break
        # medqa_embed.append(encoded)
    
    

i: 0
i: 1000
i: 2000
i: 3000
i: 4000
i: 5000
i: 6000
i: 7000
i: 8000
i: 9000
i: 10000


In [101]:

medqa_embed_np = np.array(medqa_embed)
with open('medqa_cot.npy', 'wb') as f:
    np.save(f, medqa_embed_np)


In [102]:
index.add(medqa_embed_np)


In [75]:
len(medqa_embed)

10821

In [83]:
medqa_embed[-5:]


[array([[-0.02427946,  0.10055184,  0.1260813 , ...,  0.38943747,
          0.43772724, -0.11900346],
        [-0.03789188,  0.058379  , -0.00399172, ...,  0.17896244,
          0.40114605, -0.17401543]], dtype=float32),
 array([[-0.08603792, -0.01415859, -0.03459608, ...,  0.18326746,
          0.6089421 ,  0.02165409],
        [-0.09889337,  0.19211142, -0.00448587, ...,  0.05119519,
          0.6344441 ,  0.02161592]], dtype=float32),
 array([[-0.18955734,  0.03257816, -0.171481  , ...,  0.07955323,
          0.31516275,  0.3501517 ],
        [-0.2669167 ,  0.03716958, -0.21684104, ..., -0.00108167,
          0.43806133,  0.429256  ]], dtype=float32),
 array([[-0.21115567, -0.17135051, -0.2969281 , ...,  0.21566726,
          0.12345377, -0.07346097],
        [-0.20810793, -0.12396356, -0.31118652, ...,  0.11630725,
         -0.15768407,  0.02697932]], dtype=float32),
 array([[-0.02602723, -0.1741863 , -0.36667463, ...,  0.03528364,
          0.39372545,  0.09069632],
        [-0.22

In [84]:
medqa_embed[:5]


[array([[-0.23517692,  0.22398368, -0.11469757, ...,  0.00888673,
          0.4767499 , -0.22024386],
        [-0.2586139 ,  0.37501317, -0.04517046, ..., -0.05252304,
          0.48602447, -0.44286177]], dtype=float32),
 array([[-0.1737818 ,  0.06499998, -0.24564147, ...,  0.17371292,
          0.5552193 ,  0.3111169 ],
        [-0.29244962,  0.0887769 ,  0.03712133, ...,  0.05044661,
          0.48082528,  0.28508064]], dtype=float32),
 array([[-0.13188404,  0.19391163, -0.15402895, ...,  0.03022465,
          0.2914074 , -0.03320358],
        [-0.13059261,  0.02525829, -0.1700506 , ..., -0.04651412,
          0.3095124 , -0.08072825]], dtype=float32),
 array([[-0.09424087,  0.06149313,  0.03782856, ...,  0.25264123,
          0.78810036,  0.3979171 ],
        [-0.07242909,  0.07817103,  0.01561969, ...,  0.05744653,
          0.65861195,  0.4138601 ]], dtype=float32),
 array([[-0.1109979 , -0.13378666, -0.12723249, ...,  0.12431373,
          0.2274526 ,  0.04853112],
        [-0.05

In [88]:
medqa_embed_np[:5]

array([[[-0.23517692,  0.22398368, -0.11469757, ...,  0.00888673,
          0.4767499 , -0.22024386],
        [-0.2586139 ,  0.37501317, -0.04517046, ..., -0.05252304,
          0.48602447, -0.44286177]],

       [[-0.1737818 ,  0.06499998, -0.24564147, ...,  0.17371292,
          0.5552193 ,  0.3111169 ],
        [-0.29244962,  0.0887769 ,  0.03712133, ...,  0.05044661,
          0.48082528,  0.28508064]],

       [[-0.13188404,  0.19391163, -0.15402895, ...,  0.03022465,
          0.2914074 , -0.03320358],
        [-0.13059261,  0.02525829, -0.1700506 , ..., -0.04651412,
          0.3095124 , -0.08072825]],

       [[-0.09424087,  0.06149313,  0.03782856, ...,  0.25264123,
          0.78810036,  0.3979171 ],
        [-0.07242909,  0.07817103,  0.01561969, ...,  0.05744653,
          0.65861195,  0.4138601 ]],

       [[-0.1109979 , -0.13378666, -0.12723249, ...,  0.12431373,
          0.2274526 ,  0.04853112],
        [-0.0547332 ,  0.14385927,  0.11042475, ...,  0.23039806,
        

In [96]:
data = ds["train"][1]
question = data["question"]
answer = data["response"]
# medqa_save.append([question, answer])
try:
    with torch.no_grad():
        encoded = tokenizer(
            [[question, answer]], 
            truncation=True, 
            padding=True, 
            return_tensors='pt',
            max_length=512,
        ).to(device)
        embeds = model(**encoded).last_hidden_state[:, 0, :]
        print(embeds[0].size())
        print(embeds[0])
except Exception:
    print(f"err: {i}")
    

torch.Size([768])
tensor([-2.1460e-01,  3.6954e-02, -1.7458e-01,  4.7686e-02,  3.4453e-01,
        -5.2372e-02, -2.7610e-01, -3.1338e-01,  1.1936e-02, -1.9937e-01,
         7.2273e-02,  2.0138e-01,  1.2673e-01, -3.5941e-02, -3.2551e-01,
        -1.6935e-01, -6.2313e-01, -1.2549e-01, -2.7629e-01,  3.1824e-01,
         1.0099e-01,  3.3601e-01, -1.2712e-01,  1.0168e-01,  2.1493e-01,
         3.0489e-01,  2.6947e-02,  4.0525e-03, -2.1385e-01, -1.5844e-01,
        -4.1671e-02, -1.3978e-01, -3.0609e-01, -2.4108e-02, -4.8486e-01,
        -4.8428e-02, -7.9456e-02,  2.8646e-01,  4.0283e-01, -1.0219e-02,
         4.2047e-02, -3.7425e-01, -4.2764e-02,  5.3637e-03, -8.3025e-02,
         3.8776e-01, -7.1239e-02,  6.7912e-02,  6.9406e-02, -1.4330e-01,
        -4.5817e-01,  5.6886e-02,  4.4269e-03,  6.6588e-02, -2.4175e-01,
         1.8159e-01, -4.1495e-01, -2.1175e-01, -6.3496e-01,  6.7503e-02,
        -5.3869e-03, -2.0554e-01, -2.6828e-02,  1.9761e-01, -7.9765e-03,
        -5.5975e-01, -1.4683e-01,

In [112]:

queries = [
    "How to treat diabetes?", 
]

with torch.no_grad():
    # tokenize the queries
    encoded = tokenizer(
        queries, 
        truncation=True, 
        padding=True, 
        return_tensors='pt', 
        max_length=64,
    ).to(device)
    
    # encode the queries (use the [CLS] last hidden states as the representations)
    embeds = model(**encoded).last_hidden_state[:, 0, :]
    
    # search the Faiss index
    scores, inds = index.search(embeds.cpu().numpy(), k=10)

qa = ds["train"]
# print the search results
for idx, query in enumerate(queries):
    print(f"Query: {query}")
    
    for score, ind in zip(scores[idx], inds[idx]):
        print(f"PMID: {qa[int(ind)]}; Score: {score}")

Query: How to treat diabetes with COVID-19?
PMID: {'dataset_name': 'medqa_mixtral_cot', 'system_prompt': 'You are an expert medical assistant named Aloe, developed by the High Performance Artificial Intelligence Group at Barcelona Supercomputing Center(BSC). You are to be a helpful, respectful, and honest assistant.', 'question': 'Given the following medical question with options, your task is to select the correct answer by the following process: First summarize what the question is about, then analyze each option individually, and finally select the correct answer through a step-by-step process and conclude by your final option selected. Question: A randomized controlled trial was initiated to evaluate a novel DPP-4 inhibitor for blood glucose management in diabetic patients. The study used a commonly prescribed sulfonylurea as the standard of care treatment. 2,000 patients were enrolled in the study with 1,000 patients in each arm. One of the primary outcomes was the development of 

In [109]:
ds["train"][6317]

{'dataset_name': 'medqa_mixtral_cot',
 'system_prompt': 'You are an expert medical assistant named Aloe, developed by the High Performance Artificial Intelligence Group at Barcelona Supercomputing Center(BSC). You are to be a helpful, respectful, and honest assistant.',
 'question': 'Given the following medical question with options, your task is to select the correct answer by the following process: First summarize what the question is about, then analyze each option individually, and finally select the correct answer through a step-by-step process and conclude by your final option selected. Question: A randomized controlled trial was initiated to evaluate a novel DPP-4 inhibitor for blood glucose management in diabetic patients. The study used a commonly prescribed sulfonylurea as the standard of care treatment. 2,000 patients were enrolled in the study with 1,000 patients in each arm. One of the primary outcomes was the development of diabetic nephropathy during treatment. This outc